In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [34]:
# index = 1
# def boston_url(page):
#     return f"https://results.baa.org/2023/?page={page}&pid=list&pidp=start"

# list_of_urls = []
# for page in range(1,50):
#     this_url = boston_url(page)
#     list_of_urls.append(this_url)


In [52]:
# function to get url from  gender and year 
def boston_url(year, gender):
    return f"https://results.baa.org/{year}/?page=1&event=R&event_main_group=runner&num_results=500&pid=list&pidp=start&search%5Bsex%5D={gender}&search%5Bage_class%5D=%25"

# get list of urls 
list_of_urls = []
for page in range(2018,2024):
        url1 = boston_url(page, "W")
        url2 = boston_url(page, "M")
        list_of_urls.append(url1)
        list_of_urls.append(url2)



In [53]:
# lists for name, place, finish time, sex, and year 
names = []
place = []
time = []
sex = []
year = []

for url in list_of_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    gen_text = soup.find_all('li', class_='list-group-item')[0]
    sex_curr = gen_text.find('span', class_="sex").text.strip()
    year_curr = soup.find('span', class_='hidden-xs').text.strip()
    results = soup.find_all('li', class_= 'list-group-item row')
    for result in results:
        place_elem = result.find('div', class_='list-field type-place place-primary numeric')
        place.append(place_elem.text.strip())
        name_elem = result.find('h4', class_= 'list-field type-fullname')
        names.append(name_elem.text.strip())
        time_elem = result.find_all('div', class_='split list-field type-time')
        time.append(time_elem[1].text.strip())
        sex.append(sex_curr)
        year.append(year_curr)

In [55]:
# turn into dataframe 
boston = pd.DataFrame(list(zip(names, place, time, sex, year)), 
               columns =['Name', 'Place', 'Time', 'Sex', 'Year']) 
boston.to_csv('boston_2018_2023.csv', index = False)

In [57]:
# format is slightly different for 2017 and before
list_of_urls = []
for page in range(2012,2018):
        url1 = boston_url(page, "W")
        url2 = boston_url(page, "M")
        list_of_urls.append(url1)
        list_of_urls.append(url2)

In [58]:
data_all = pd.DataFrame() # initialize df 

for url in list_of_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # gender = soup.find('div', class_="list-info-text").find('span', class_="sex").text.strip()
    gender = re.findall("(?<=5D=)(.*)(?=&search)", url)
    year = re.findall("(?<=baa.org/)(.*)(?=\/\?page)",url)
    table = soup.find('table', class_='list-table')
    columns = []
    data = []

# Extract column names from the table header (th elements)
    header_row = table.find('thead').find('tr')
    for header_cell in header_row.find_all('th'):
        columns.append(header_cell.text.strip())

# Extract data rows from the table body (td elements)
    body = table.find('tbody')
    for row in body.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        data.append(row_data)
    df = pd.DataFrame(data, columns = columns)
    yearcol = year * len(df)
    sexcol = gender * len(df)
    df['year'] = yearcol 
    df['sex'] = sexcol

    data_all = data_all.append(df, ignore_index = True)



/Users/lilykoffman/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [59]:
data_all.to_csv('boston_2012_2017.csv', index = False)

In [101]:
url = "https://berlin.r.mikatiming.com/2023/?page=4&pid=list"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser') 
tab_rows = soup.find_all('li', class_="list-active list-group-item row")


for item in tab_rows:
    place = item.find('div', class_="list-field type-place place-primary numeric")
    place_second = item.find('div', class_="list-field type-place place-secondary hidden-xs numeric")
    name = item.find('h4', class_="list-field type-fullname")
    time_div = item.find_all('div', class_="visible-xs-block visible-sm-block list-label")
    print(time_div)
    # element 02:11:09
    # //*[@id="cbox-main"]/div[2]/ul/li[2]/div[2]/div[2]/div/div[2]/text()



[<div class="visible-xs-block visible-sm-block list-label">Number</div>, <div class="visible-xs-block visible-sm-block list-label">AC</div>, <div class="visible-xs-block visible-sm-block list-label">Club</div>, <div class="visible-xs-block visible-sm-block list-label"></div>]
[<div class="visible-xs-block visible-sm-block list-label">Number</div>, <div class="visible-xs-block visible-sm-block list-label">AC</div>, <div class="visible-xs-block visible-sm-block list-label">Club</div>, <div class="visible-xs-block visible-sm-block list-label"></div>]
[<div class="visible-xs-block visible-sm-block list-label">Number</div>, <div class="visible-xs-block visible-sm-block list-label">AC</div>, <div class="visible-xs-block visible-sm-block list-label">Club</div>, <div class="visible-xs-block visible-sm-block list-label"></div>]
[<div class="visible-xs-block visible-sm-block list-label">Number</div>, <div class="visible-xs-block visible-sm-block list-label">AC</div>, <div class="visible-xs-block

In [130]:
def chi_url(page, year):
    return f"https://chicago-history.r.mikatiming.com/{year}/?page={page}&lang=EN_CAP&pid=list"

# get list of urls 
list_of_urls = []
for year in range(2012, 2024):
    for page in range(1, 2):
        url = chi_url(page, year)
        list_of_urls.append(url)

In [162]:
places_s = []
places_p = []
names = []
times = []
bibs = []
divs = []
years = []

for url in list_of_urls: 
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser') 
    tab_rows = soup.find_all('li', class_="list-active")
    year_curr = re.findall("(?<=mikatiming.com/)(.*)(?=\/\?page)", url)[0]
    for row in tab_rows:
        place_s = row.find('div', class_="list-field type-place place-secondary hidden-xs numeric")
        places_s.append(place_s.text.strip())
        place_p = row.find('div', class_="list-field type-place place-primary numeric")
        places_p.append(place_p.text.strip())
        name = row.find('h4', class_="list-field type-fullname")
        names.append(name.text.strip())
        time = row.find('div', class_="list-field type-time")
        times.append(time.text.strip())
        bib = row.find('div', class_="list-field type-field")
        bibs.append(bib.text.strip())
        divis = row.find('div', class_="list-field type-age_class")
        divs.append(divis.text.strip())
        years.append(year_curr)

             

In [163]:
chi  = pd.DataFrame(list(zip(names, places_s, places_p, times, bibs, divs, year)),
               columns =['Name', 'Place S', 'Place P', 'Time', 'Bib', 'Division']) 



In [164]:
chi

,Name,Place S,Place P,Time,Bib,Division
0,"Hinton, James (USA)",1,1,Finish04:09:21,BIB18990,Division–
1,"Tura Abdiwak, Seifu (ETH)",1,1,Finish02:06:12,BIB3,Division20-24
2,"Labus, Kathy (USA)",3,1,Finish04:42:52,BIB18991,Division–
3,"Chepngetich, Ruth (KEN)",21,1,Finish02:22:31,BIB102,Division25-29
4,"Hinton, Brian (USA)",2,2,Finish04:09:23,BIB18989,Division–
...,...,...,...,...,...,...
151,"Kiptanui, Eric (KEN)",3,3,Finish02:06:51,BIB7,Division30-34
152,"Hall, Sara (USA)",41,3,Finish02:27:19,BIB103,Division35-39
153,"Sayih, Michael (USA)",5,4,Finish04:44:56,BIB18986,Division–
154,"Suzuki, Kengo (JPN)",4,4,Finish02:08:50,BIB5,Division25-29


In [ ]:
def london_url(page, year):
    return f"https://results.tcslondonmarathon.com/{year}/?page={page}&pid=list"
# get list of urls 
list_of_urls = []
for year in range(2012, 2024):
    for page in range(1, 50):
        url = london_url(page, year)
        list_of_urls.append(url)

In [ ]:
url = list_of_urls[0]
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser') 
tab_rows = soup.find_all('li', class_="list-active")
year = re.findall("(?<=mikatiming.com/)(.*)(?=\/\?page)", url)
for row in tab_rows:
    place_s = row.find('div', class_="list-field type-place place-secondary hidden-xs numeric")
    places_s.append(place_s.text.strip())
    place_p = row.find('div', class_="list-field type-place place-primary numeric")
    places_p.append(place_p.text.strip())
    name = row.find('h4', class_="list-field type-fullname")
    names.append(name.text.strip())
    time = row.find('div', class_="list-field type-time")
    times.append(time.text.strip())
    bib = row.find('div', class_="list-field type-field")
    bibs.append(bib.text.strip())
    divis = row.find('div', class_="list-field type-age_class")
    divs.append(divis.text.strip())
    years.append(year)